Applying convolution to Text -Chapter 9

Detecting sarcasm in text using CNNs

In [1]:
import pandas as pd
import numpy as np
import re
import json
import gensim
import math
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import KeyedVectors
import keras
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Dense, Dropout, Conv1D, GlobalMaxPooling1D
import h5py


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
import json

def parse_data(file):
    for line in open(file, 'r'):
        try:
            yield json.loads(line)
        except json.JSONDecodeError as e:
            print(f"Error in line: {line}")
            raise e

try:
    data = list(parse_data('/content/Sarcasm_Headlines_Dataset_v2.json'))
    df = pd.DataFrame(data)
except json.JSONDecodeError as e:
    print(e)


In [3]:
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [4]:
df.shape

(28619, 3)

In [5]:
df.drop(columns="article_link")

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...
...,...,...
28614,1,jews to celebrate rosh hashasha or something
28615,1,internal affairs investigator disappointed con...
28616,0,the most beautiful acceptance speech this week...
28617,1,mars probe destroyed by orbiting spielberg-gat...


In [6]:
classes = np.unique(np.array(df['is_sarcastic']))
classes

array([0, 1])

In [7]:
def text_clean(corpus):
    '''
    Purpose : Function to keep only alphabets, digits and certain words (punctuations, qmarks, tabs etc. removed)

    Input : Takes a text corpus, 'corpus' to be cleaned along with a list of words, 'keep_list', which have to be retained
            even after the cleaning process

    Output : Returns the cleaned text corpus

    '''
    cleaned_corpus = pd.Series()
    for row in corpus:
        qs = []
        for word in row.split():
            p1 = re.sub(pattern='[^a-zA-Z0-9]',repl=' ',string=word)
            p1 = p1.lower()
            qs.append(p1)
        cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
    return cleaned_corpus

In [8]:
def stopwords_removal(corpus):
    stop = set(stopwords.words('english'))
    corpus = [[x for x in x.split() if x not in stop] for x in corpus]
    return corpus

In [9]:
def lemmatize(corpus):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    return corpus

In [10]:
def stem(corpus, stem_type = None):
    if stem_type == 'snowball':
        stemmer = SnowballStemmer(language = 'english')
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    else :
        stemmer = PorterStemmer()
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    return corpus

In [11]:

def preprocess(corpus, cleaning = True, stemming = False, stem_type = None, lemmatization = False, remove_stopwords = True):

    '''
    Purpose : Function to perform all pre-processing tasks (cleaning, stemming, lemmatization, stopwords removal etc.)

    Input :
    'corpus' - Text corpus on which pre-processing tasks will be performed

    'cleaning', 'stemming', 'lemmatization', 'remove_stopwords' - Boolean variables indicating whether a particular task should
                                                                  be performed or not
    'stem_type' - Choose between Porter stemmer or Snowball(Porter2) stemmer. Default is "None", which corresponds to Porter
                  Stemmer. 'snowball' corresponds to Snowball Stemmer

    Note : Either stemming or lemmatization should be used. There's no benefit of using both of them together

    Output : Returns the processed text corpus

    '''
    if cleaning == True:
        corpus = text_clean(corpus)

    if remove_stopwords == True:
        corpus = stopwords_removal(corpus)
    else :
        corpus = [[x for x in x.split()] for x in corpus]

    if lemmatization == True:
        corpus = lemmatize(corpus)


    if stemming == True:
        corpus = stem(corpus, stem_type)

    corpus = [' '.join(x) for x in corpus]


    return corpus

In [12]:
df['headline']

0        thirtysomething scientists unveil doomsday clo...
1        dem rep. totally nails why congress is falling...
2        eat your veggies: 9 deliciously different recipes
3        inclement weather prevents liar from getting t...
4        mother comes pretty close to using word 'strea...
                               ...                        
28614         jews to celebrate rosh hashasha or something
28615    internal affairs investigator disappointed con...
28616    the most beautiful acceptance speech this week...
28617    mars probe destroyed by orbiting spielberg-gat...
28618                   dad clarifies this not a food stop
Name: headline, Length: 28619, dtype: object

In [13]:
headlines = preprocess(df['headline'], lemmatization = True, remove_stopwords = True)

Streaming output truncated to the last 5000 lines.
<ipython-input-7-b216062b07a1>:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
<ipython-input-7-b216062b07a1>:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
<ipython-input-7-b216062b07a1>:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
<ipython-input-7-b216062b07a1>:18: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
<ipython-

In [14]:
headlines[0:5]

['thirtysomething scientists unveil doomsday clock hair loss',
 'dem rep totally nail congress fall short gender racial equality',
 'eat veggies 9 deliciously different recipes',
 'inclement weather prevent liar get work',
 'mother come pretty close use word stream correctly']

Loading Word2vec model

In [15]:
#!pip install gensim
import gensim
from gensim.models import KeyedVectors
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [16]:
model=KeyedVectors.load_word2vec_format('/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz',binary=True)

Defining model Parameters


In [17]:
MAX_LENGTH = 10
VECTOR_SIZE = 300

Data vectorization and standarization

we will be sending the word vectors, each of size 300, to
our CNN model. However, for that, our data dimensions should be
standardized. Since our data is from headlines, we will standardize them so that
they have a length of 10. If any of the headlines contain more than 10 characters,
they will be subsampled to keep the word embeddings for the first 10 characters.
If any of the headlines contain less than 10 characters, we will pad them so that
they have vectors with values of 0.


In [21]:
def vectorize_data(data, model):
    vectors = []
    padding_vector = [0.0] * VECTOR_SIZE

    for data_point in data:
        data_point_vectors = []
        count = 0

        tokens = data_point.split()

        for token in tokens:
            if count >= MAX_LENGTH:
                break
            if token in model.key_to_index:  # Changed from model.wv.key_to_index
                data_point_vectors.append(model[token])  # Changed from model.wv[token]
            count += 1

        # Padding the remaining vectors if the data point is shorter than MAX_LENGTH
        data_point_vectors += [padding_vector] * (MAX_LENGTH - len(data_point_vectors))

        vectors.append(data_point_vectors)

    return vectors


In [22]:
vectorized_headlines = vectorize_data(headlines,model)

 Let's add a small validation to ensure that the 10 vectors are present for each
headline, as defined in the MAX_LENGTH parameter:


In [23]:
for i, vec in enumerate(vectorized_headlines):
 if len(vec) != MAX_LENGTH:
  print(i)


In [24]:
len(vectorized_headlines[i])

10

In [25]:
train_div = math.floor(0.7 * len(vectorized_headlines))
train_div

20033

In [26]:
X_train = vectorized_headlines[:train_div]
y_train = df['is_sarcastic'][:train_div]
X_test = vectorized_headlines[train_div:]
y_test = df['is_sarcastic'][train_div:]

print('The size of X_train is:', len(X_train), '\nThe size of y_train is:', len(y_train),
      '\nThe size of X_test is:', len(X_test), '\nThe size of y_test is:', len(y_test))

The size of X_train is: 20033 
The size of y_train is: 20033 
The size of X_test is: 8586 
The size of y_test is: 8586


In [27]:

X_train = np.reshape(X_train, (len(X_train), MAX_LENGTH, VECTOR_SIZE))
X_test = np.reshape(X_test, (len(X_test), MAX_LENGTH, VECTOR_SIZE))
y_train = np.array(y_train)
y_test = np.array(y_test)


Defining Neural Network Model Parameters

In [28]:
FILTERS=8
KERNEL_SIZE=3
HIDDEN_LAYER_1_NODES=10
HIDDEN_LAYER_2_NODES=5
DROPOUT_PROB=0.35
NUM_EPOCHS=10
BATCH_SIZE=50

In [29]:
model = Sequential()

model.add(Conv1D(FILTERS,
                 KERNEL_SIZE,
                 padding='same',
                 strides=1,
                 activation='relu',
                 input_shape = (MAX_LENGTH, VECTOR_SIZE)))
model.add(GlobalMaxPooling1D())
model.add(Dense(HIDDEN_LAYER_1_NODES, activation='relu'))
model.add(Dropout(DROPOUT_PROB))
model.add(Dense(HIDDEN_LAYER_2_NODES, activation='relu'))
model.add(Dropout(DROPOUT_PROB))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 10, 8)             7208      
                                                                 
 global_max_pooling1d (Glob  (None, 8)                 0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 10)                90        
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 dropout_1 (Dropout)         (None, 5)                 0         
                                                        

In [30]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [31]:
training_history = model.fit(X_train, y_train, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/10
401/401 [==============================] - 3s 4ms/step - loss: 0.6669 - accuracy: 0.5929
Epoch 2/10
401/401 [==============================] - 2s 4ms/step - loss: 0.5901 - accuracy: 0.6988
Epoch 3/10
401/401 [==============================] - 2s 4ms/step - loss: 0.5398 - accuracy: 0.7439
Epoch 4/10
401/401 [==============================] - 2s 4ms/step - loss: 0.5042 - accuracy: 0.7721
Epoch 5/10
401/401 [==============================] - 2s 5ms/step - loss: 0.4805 - accuracy: 0.7876
Epoch 6/10
401/401 [==============================] - 2s 6ms/step - loss: 0.4623 - accuracy: 0.7981
Epoch 7/10
401/401 [==============================] - 2s 4ms/step - loss: 0.4473 - accuracy: 0.8087
Epoch 8/10
401/401 [==============================] - 2s 4ms/step - loss: 0.4300 - accuracy: 0.8165
Epoch 9/10
401/401 [==============================] - 2s 4ms/step - loss: 0.4201 - accuracy: 0.8221
Epoch 10/10
401/401 [==============================] - 2s 4ms/step - loss: 0.4040 - accuracy: 0.8351

Model Evaluation

In [32]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.7495
